# WFI Reference File Pipeline (RFP) - Read Noise Example
This notebook shows how to use the WFI Reference File Pipeline (RFP) to create a readnoise reference file. The RFP will simulate dark reads that will be used as input into the ReadNoise module. 

Additional functionality such as reading a list of files to construct the input data, and optional methods to calculate the read noise are in development and only mentioned, but are not necessary to generate reference files suitable for delivery to CRDS.

In [ ]:
# Imports
import numpy as np

from wfi_reference_pipeline.constants import (
    DETECTOR_PIXEL_X_COUNT,
    DETECTOR_PIXEL_Y_COUNT,
)
from wfi_reference_pipeline.reference_types.readnoise.readnoise import ReadNoise
from wfi_reference_pipeline.resources.make_dev_meta import MakeDevMeta
from wfi_reference_pipeline.utilities.simulate_reads import simulate_dark_reads

#### Step 1: Setup

In [ ]:
# Output file path
outfile = '/PATH/TO/scratch/rfp_readnoise_dev_file.asdf'

# Use dev meta maker
tmp = MakeDevMeta(ref_type='READNOISE')

# Create a user input rate image
user_rate_image = np.random.normal(loc=5, scale=1, size=(DETECTOR_PIXEL_X_COUNT, DETECTOR_PIXEL_Y_COUNT)).astype(np.float32)

#### Step 2: Instantiate the Read Noise object with user input rate image

In [23]:
# Instantiate the Read Noise object
rfp_readnoise = ReadNoise(
    meta_data=tmp.meta_readnoise,
    ref_type_data=user_rate_image,
    outfile=outfile,
    clobber=True
)

# Save the file
rfp_readnoise.generate_outfile()

#### Note that the meta object can have specific meta fields modified before sending to RFP module to instantiate. The meta data object is converted to a dictionary when the file is being written in the generate_outfile() abstract method in the ReferenceType base class. This can be seen by inspecting the meta data object where the key _use_after is an object property not yet in the correct astropy date time format.

In [27]:
det = 5  # Make the readnoise reference file for WFI05.
tmp.meta_readnoise.instrument_detector = 'WFI' + f"{det:02d}"
tmp.meta_readnoise.useafter = '2024-04-22T00:00:00.000'  # Update useafter to the new string
tmp.meta_readnoise

WFIMetaReadNoise(reference_type='READNOISE', pedigree='DUMMY', description='For RFP Development and DMS Build Support.', author='RFP', _use_after=<Time object: scale='utc' format='isot' value=2023-01-01T00:00:00.000>, telescope='ROMAN', origin='STSCI', instrument='WFI', instrument_detector='WFI05')

### Create Read Noise with a Simulated Dark Cube
This section demonstrates how to create a Read Noise reference file using a simulated dark data cube.

In [ ]:
# Output file path
outfile = '/PATH/TO/scratch/rfp_readnoise_dev_file.asdf'

# Use dev meta maker
tmp = MakeDevMeta(ref_type='READNOISE')

# Set the simulated readnoise variance
sim_readnoise_var = 5

#### Step 3: Create Simulated Data Cube

In [17]:
# Create simulated data cube
sim_dev_cube, dev_rate_image = simulate_dark_reads(40, dark_rate=1, noise_mean=15, noise_var=np.sqrt(sim_readnoise_var))

Making WFI Imaging Mode (WIM) dark read cube with exposure time 3.04 seconds.


#### Step 4: Instantiate and create Read Noise object with data cube

In [20]:
# Instantiate the ReadNoise object
rfp_readnoise = ReadNoise(
    meta_data=tmp.meta_readnoise,
    ref_type_data=sim_dev_cube,
    outfile=outfile,
    clobber=True
)

# Generate rate image from data cube
rfp_readnoise.make_rate_image_from_data_cube()

# Generate readnoise image from data cube
rfp_readnoise.make_readnoise_image()

# Save the file
rfp_readnoise.generate_outfile()